In [6]:
!pip install -q transformers

In [7]:
import pandas as pd
from tqdm import tqdm


In [8]:
train=pd.read_csv("/kaggle/input/tweeter/train.csv")
test=pd.read_csv("/kaggle/input/tweeter/test.csv")
val=pd.read_csv("/kaggle/input/tweeter/val.csv")

In [10]:
y_train = train['Category']
y_val = val['Category']
y_test = test['Category']
X_train = train['Text']
X_val = val['Text']
X_test = test['Text']

In [12]:
def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [13]:
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def preprocessing_for_bert(data):
    input_ids = []
    attention_masks = []
    for sentence in tqdm(data):
        encoded_sent = bert_tokenizer.encode_plus(
            text=text_preprocessing(sentence),  
            add_special_tokens=True,        
            max_length=MAXIMUM_LENGTH,                  
            pad_to_max_length=True,         
            return_attention_mask=True      
            )

        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
import regex as re
MAXIMUM_LENGTH = 64

token_ids = list(preprocessing_for_bert([X_train[0]])[0].squeeze().numpy())
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

  0%|          | 0/1 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 91.18it/s]


Original:  Disnleyland ISN'T tha happiest place in tha world! Grad trip can suck my cock, well maybe not magic mountain &amp; its a small world ride 
Token IDs:  [101, 4487, 2015, 20554, 3240, 3122, 3475, 1005, 1056, 22794, 5292, 9397, 10458, 2173, 1999, 22794, 2088, 999, 24665, 4215, 4440, 2064, 11891, 2026, 10338, 1010, 2092, 2672, 2025, 3894, 3137, 1004, 2049, 1037, 2235, 2088, 4536, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenizing data...


100%|██████████| 240000/240000 [00:44<00:00, 5408.35it/s]


In [16]:
y_train.replace(4,1,inplace= True)
y_val.replace(4,1, inplace=True)

In [17]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_val.values)
batch_size = 512

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


validation_data = TensorDataset(val_inputs, val_masks, val_labels)
validation_sampler = SequentialSampler(validation_data)
vaidation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [18]:
import torch
import torch.nn as nn
from transformers import BertModel

class BERT_Classification(nn.Module):
    def __init__(self, bert_freezing=False):
        super(BERT_Classification, self).__init__()
        Dimensions_input, H, Dimensions_out = 768, 50, 2
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Sequential(
            nn.Linear(Dimensions_input, H),
            nn.ReLU(),
            nn.Linear(H, Dimensions_out)
        )

        if bert_freezing:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):

        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        last_hidden_state_cls = outputs[0][:, 0, :]

        logits = self.classifier(last_hidden_state_cls)

        return logits

In [19]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(num_epochs=4):
    bert_classification_model = BERT_Classification(bert_freezing=False)
    bert_classification_model= nn.DataParallel(bert_classification_model)
    bert_classification_model.to(device)
    optimizer = AdamW(bert_classification_model.parameters(),
                      learning_rate=5e-5,    
                      eps=1e-8   
                      )
    total_steps = len(train_dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, 
                                                num_training_steps=total_steps)
    return bert_classification_model, optimizer, scheduler

In [20]:
import random
import time
import numpy as np
loss_fn = nn.CrossEntropyLoss()
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def Train(model, train_dataloader, vaidation_dataloader=None, num_epochs=4, evaluation=False):
    for epoch_i in range(num_epochs):
        t0_epoch, t0_batch = time.time(), time.time()
        total_loss, batch_loss, batch_counts = 0, 0, 0
        for step, batch in enumerate(tqdm(train_dataloader)):
            batch_counts +=1 # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
            model.zero_grad()
            logits = model(b_input_ids, b_attn_mask)
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
            time_elapsed = time.time() - t0_batch
            batch_loss, batch_counts = 0, 0
            t0_batch = time.time()
            avg_train_loss = total_loss / len(train_dataloader)
    if evaluation == True:
        validation_loss, validation_accuracy = evaluate(model, vaidation_dataloader)
        time_elapsed = time.time() - t0_epoch
        
def evaluate(model, vaidation_dataloader):
    model.eval()
    validation_accuracy = []
    validation_loss = []
    for batch in vaidation_dataloader:
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
            loss = loss_fn(logits, b_labels)
            validation_loss.append(loss.item())
            preds = torch.argmax(logits, dim=1).flatten()
            accuracy = (preds == b_labels).cpu().numpy().mean() * 100
            validation_accuracy.append(accuracy)
    validation_loss = np.mean(validation_loss)
    validation_accuracy = np.mean(validation_accuracy)
    return validation_loss, validation_accuracy

In [21]:
set_seed(42) 
bert_classification_model, optimizer, scheduler = initialize_model(num_epochs=1)
Train(bert_classification_model, train_dataloader, vaidation_dataloader, num_epochs=1, evaluation=True)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------


  1%|          | 21/2188 [00:50<1:21:03,  2.24s/it]

   1    |   20    |   0.548140   |     -      |     -     |   50.22  
----------------------------------------------------------------------


...

100%|█████████▉| 2181/2188 [1:27:00<00:16,  2.39s/it]

   1    |  2180   |   0.297026   |     -      |     -     |   47.81  
----------------------------------------------------------------------


100%|██████████| 2188/2188 [1:27:16<00:00,  2.39s/it]

   1    |  2187   |   0.296606   |     -      |     -     |   15.70  
----------------------------------------------------------------------


   1    |    -    |   0.320093   |  0.297008  |   87.30   |  5657.63 
----------------------------------------------------------------------


Training complete!


In [22]:
torch.save(bert_classification_model, './bert.pt')

In [23]:
y_test.replace(4,1,inplace=True)
test_labels = torch.tensor(y_test.values)
test_inputs, test_masks = preprocessing_for_bert(X_test)
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

  0%|          | 0/240000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 240000/240000 [00:46<00:00, 5107.75it/s]


In [24]:

loss,acc = evaluate(bert_classification_model, test_dataloader)

0.29630521765904133 87.28969549573561


In [26]:
print("The Loss Value is "+str(loss)+" .The accuracy values are "+str(acc))

The Loss Value is 0.29630521765904133 .The accuracy values are 87.28969549573561
